api from https://banks.data.fdic.gov/docs/#/Structure/searchInstitutions

based on work from
https://doi.org/10.1016/j.ribaf.2017.07.104 - Predicting bank failure: An improvement by implementing a machine-learning approach to classical financial ratios
https://doi.org/10.1016/j.dss.2012.11.015 - Partial Least Square Discriminant Analysis for bankruptcy prediction
https://doi.org/10.1016/j.eswa.2008.01.053 - Effects of feature construction on classification performance: An empirical study in bank failure prediction

In [2]:
import requests
import pandas as pd
from io import StringIO
from urllib.parse import quote_plus

# pd.set_option('display.max_rows', None)

## Construct API call

In [98]:
# function to get data from BankFind API
def getData(url: str, filter: str, fields: str, sortby: str = 'CERT', order: str = 'ASC', n: int = 10000, k: int = 0, suffix: str = '&format=csv&download=false&filename=data_file') -> pd.DataFrame:
    request = requests.get(url + 'filters=' + quote_plus(filter) + '&fields=' + quote_plus(fields) + '&sort_by=' + sortby + '&sort_order=' + order + '&limit=' + str(n) + '&offset=' + str(k) + suffix).text
    return pd.read_csv(StringIO(request))

In [121]:
# function to get data from BankFind API
def getData2(url: str, filter: str, fields: str, sortby: str = 'CERT', order: str = 'ASC', n: int = 10000, k: int = 0, suffix: str = '&format=csv&download=false&filename=data_file') -> pd.DataFrame:
    request = requests.get(url + 'filters=' + quote_plus(filter) + '&fields=' + quote_plus(fields) + '&sort_by=' + sortby + '&sort_order=' + order + '&limit=' + str(n) + '&offset=' + str(k) + suffix).text
    return StringIO(request)

In [29]:
# reset index and drop redundant columns for bank dataframe
def cleanbankdata(df: pd.DataFrame) -> None:
    df.set_index('CERT', inplace=True)
    df.drop('ID', axis = 1, inplace = True)

### Get lists of all banks 2006-present

In [122]:
n = 50

In [174]:
# strings for getting lists of active and failed banks
institutionurl = 'https://banks.data.fdic.gov/api/institutions?'
filtersfailed = 'ACTIVE:0 AND DATEUPDT:[\"2006-01-01\" TO \"2023-12-31\"]'
filtersactive = 'ACTIVE:1 AND DATEUPDT:[\"2006-01-01\" TO \"2023-12-31\"]'
bankfields = 'STALP,NAME,ACTIVE,CERT,DATEUPDT'


# get failed banks
failedbanks = getData(institutionurl, filtersfailed, bankfields, n = n)
cleanbankdata(failedbanks)

# get active banks
activebanks = getData(institutionurl, filtersactive, bankfields, n = n)
cleanbankdata(activebanks)

In [100]:
failedbanks.head()

,ACTIVE,DATEUPDT,NAME,STALP
CERT,,,,
264,0,04/27/2012,Farmers National Bank,KY
266,0,01/11/2019,Farmers Deposit Bank,KY
278,0,01/13/2006,Ascencia Bank,KY
310,0,12/12/2017,The Highlands Bank,LA
313,0,04/13/2015,American Gateway Bank,LA


In [101]:
activebanks.head()

,ACTIVE,DATEUPDT,NAME,STALP
CERT,,,,
14,1,06/05/2023,State Street Bank and Trust Company,MA
35,1,09/02/2022,AuburnBank,AL
39,1,03/28/2023,Robertson Banking Company,AL
41,1,08/31/2022,Phenix-Girard Bank,AL
49,1,08/31/2022,Bank of Evergreen,AL


In [115]:
print(f'number of failed banks: {len(failedbanks)}')
print(f'number of active banks: {len(activebanks)}')

number of failed banks: 250
number of active banks: 250


### Get financials of banks

In [156]:
# financials

# gets financial data for all banks currently active from their year-end report in 2021
financialsurl = 'https://banks.data.fdic.gov/api/financials?'
# filters = ['ACTIVE:1', 'REPYEAR:[2000 TO 2023]']
filters = 'RISDATE:20211231'
featureslist =  ['NAME,RISDATE,CERT,REPYEAR,',
                'LNATRESR,ELNLOS,NIM,EAMINTAN,LNLSGRS,NTLNLS,EQ,AVASSETJ'] #,
                # 'RBCT1,EQTOTR,EQV,LNLSNET,LIAB,LIABEQR,LIABEQ,DEP,',
                # 'NIMY,NIMR,NONIXR,PTAXNETINCR,ITAX,',
                # 'ROA,ROE,NETINC,EEFFR,CHBAL']
features = ''.join(featureslist)

featurenames = {'LNATRESR': 'LOAN LOSS RESERVE/GROSS LN&LS',
                'ELNLOS' : 'PROVISIONS FOR LN & LEASE LOSSES',
                'NIM' : 'NET INTEREST INCOME',
                'EAMINTAN' : 'AMORT & IMPAIR LOSS AST',
                'LNLSGRS' : 'LOANS AND LEASES, GROSS',
                'NTLNLS' : 'TOTAL LN&LS NET CHARGE-OFFS',
                'EQ' : 'Equity Capital',
                'AVASSETJ' : 'AVERAGE ASSETS-ADJUSTED-PCA',
                'RBCT1' : 'TIER 1 RBC-PCA',
                'EQTOTR' : 'TOTAL EQUITY CAPITAL RATIO',
                'EQV' : 'BANK EQUITY CAPITAL/ASSETS',
                'LNLSNET' : 'LOANS AND LEASES-NET',
                'CUSLI' : 'CUSTOMERS ACCEPTANCES',
                'LIAB' : 'TOTAL LIABILITIES',
                'LIABEQR' : 'TOTAL LIABILITIES & CAPITAL RATIO',
                'LIABEQ' : 'TOTAL LIABILITIES & CAPITAL',
                'DEP' : 'Total deposits',
                'NIMY' : 'NET INTEREST MARGIN',
                'NIMR' : 'NET INTEREST INCOME RATIO',
                'IOTHFEE' : 'OTHER FEE INCOME',
                'NONIXR' : 'TOTAL NONINTEREST EXPENSE RATIO',
                'PTAXNETINCR' : 'PRE-TAX NET INCOME OPERATING INCOME RATIO',
                'ITAX' : 'APPLICABLE INCOME TAXES',
                'ROA' : 'Return on assets (ROA)',
                'ROE' : 'Return on equity (ROE)',
                'NETINC' : 'Net income',
                'EEFFR' : 'EFFICIENCY RATIO',
                'CHBAL' : 'CASH & DUE FROM DEPOSITORY INST'

                }

activefinancials = getData(financialsurl, filters, features)

In [157]:
activefinancials.head()

,AVASSETJ,CERT,EAMINTAN,ELNLOS,EQ,ID,LNATRESR,LNLSGRS,NAME,NIM,NTLNLS,REPYEAR,RISDATE
0,290403000.0,14,243000.0,-29000.0,27821000.0,14_20211231,0.266454,32654000,STATE STREET BANK&TRUST CO,1996000.0,2000.0,2021,20211231
1,1070188.0,35,0.0,-600.0,100951.0,35_20211231,1.074301,460499,AUBURNBANK,24035.0,79.0,2021,20211231
2,410344.0,39,0.0,300.0,36189.0,39_20211231,1.574606,267178,ROBERTSON BANKING CO,12091.0,-8.0,2021,20211231
3,277748.0,41,0.0,100.0,36064.0,41_20211231,1.498753,74195,PHENIX-GIRARD BANK,6880.0,41.0,2021,20211231
4,384528.0,46,0.0,24.0,35665.0,46_20211231,1.259482,190237,MERCHANTS BANK OF ALABAMA,11151.0,45.0,2021,20211231


In [62]:
print(f'number of financial reports (banks): {len(activefinancials)}')

number of financial reports (banks): 4904


In [63]:
activedata = activebanks.merge(activefinancials, on = 'CERT', how = 'left', suffixes=['_b', '_f'])

In [64]:
activedata[['NAME_b', 'NAME_f', 'CERT']].head(5)

,NAME_b,NAME_f,CERT
0,State Street Bank and Trust Company,STATE STREET BANK&TRUST CO,14
1,AuburnBank,AUBURNBANK,35
2,Robertson Banking Company,ROBERTSON BANKING CO,39
3,Phenix-Girard Bank,PHENIX-GIRARD BANK,41
4,Bank of Evergreen,BANK OF EVERGREEN,49


In [65]:
activedata.describe()

,CERT,ACTIVE,AVASSETJ,CHBAL,DEP,EAMINTAN,EEFFR,ELNLOS,EQ,EQTOTR,...,NIM,NIMR,NIMY,NONIXR,NTLNLS,PTAXNETINCR,RBCT1,RISDATE,ROA,ROE
count,10.000000,10.0,1.000000e+01,1.000000e+01,1.000000e+01,10.000000,10.000000,10.000000,1.000000e+01,10.000000,...,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000e+01,10.0,10.000000,10.000000
mean,44.300000,1.0,2.943238e+07,1.097573e+07,2.642041e+07,24300.000000,63.330700,-2677.800000,2.825222e+06,11.577759,...,2.104419e+05,2.772720,2.999174,2.408211,207.400000,1.358282,1.927120e+06,20211231.0,1.242324,10.806000
std,12.841339,0.0,9.169636e+07,3.455537e+07,8.235493e+07,76843.347142,9.845532,9265.074645,8.782673e+06,3.220079,...,6.274425e+05,0.854979,0.914735,0.433666,654.145957,0.486909,5.944413e+06,0.0,0.453955,4.377856
min,14.000000,1.0,7.504400e+04,2.971000e+03,6.657600e+04,0.000000,44.434749,-29000.000000,8.994000e+03,8.779710,...,2.205000e+03,0.632477,0.776841,1.576364,-437.000000,0.868488,8.959000e+03,20211231.0,0.729525,5.700000
25%,39.500000,1.0,2.358605e+05,2.514550e+04,1.837470e+05,0.000000,56.806308,0.500000,3.064600e+04,9.289024,...,6.898500e+03,2.595117,2.712843,2.293963,-2.750000,0.926453,3.066175e+04,20211231.0,0.766624,7.815000
50%,49.500000,1.0,3.282150e+05,5.390850e+04,2.783005e+05,0.000000,68.218648,70.000000,4.115800e+04,10.830787,...,9.389000e+03,3.078783,3.315969,2.477719,29.500000,1.217548,4.135000e+04,20211231.0,1.266340,9.930000
75%,51.750000,1.0,8.751250e+05,7.573950e+04,7.443518e+05,0.000000,70.147955,257.500000,8.662125e+04,12.638750,...,2.104900e+04,3.181966,3.485263,2.617633,112.000000,1.667964,8.609025e+04,20211231.0,1.651537,12.472500
max,58.000000,1.0,2.904030e+08,1.093220e+08,2.608050e+08,243000.000000,73.829953,1493.000000,2.782100e+07,19.467107,...,1.996000e+06,3.757897,4.130662,3.164666,2000.000000,2.210703,1.884500e+07,20211231.0,1.876296,18.880000


In [38]:
activedata.dropna(inplace= True)

In [39]:
len(activedata)

4587

In [189]:
# create a new column containing one year prior to the year of failure

failedbanks['prevYr'] = failedbanks.DATEUPDT.str.rsplit('/', expand = True, n = 1)[1].astype(int) - 1
failedbanks['targetdate'] = failedbanks.prevYr * 10000 + 1231


In [190]:
failedbanks

,ACTIVE,DATEUPDT,NAME,STALP,prevYr,targetdate
CERT,,,,,,
9,0,02/27/2008,Union Trust Company,ME,2007,20071231
46,0,01/07/2022,Merchants Bank of Alabama,AL,2021,20211231
47,0,07/10/2020,Traders & Farmers Bank,AL,2019,20191231
57,0,11/21/2006,Community Bank,AL,2005,20051231
59,0,01/09/2013,The Citizens Bank,AL,2012,20121231
68,0,03/27/2014,Peachtree Bank,AL,2013,20131231
91,0,09/26/2016,Allied Bank,AR,2015,20151231
92,0,10/27/2014,Delta Trust & Bank,AR,2013,20131231
101,0,12/03/2019,Bank of Prescott,AR,2018,20181231


In [127]:
# financials of failed banks in the year prior to failure

failedfinancialslist = []
for bankID in failedbanks.index:
    year = str(failedbanks.prevYr[bankID])
    filter = f'RISDATE:{year}1231 AND CERT:{str(bankID)}'
    try:
        failedfinancialslist.append(getData2(financialsurl, filter, features, n = 1))
    except:
        pass



In [119]:
failedfinancials = pd.read_csv(failedfinancialslist)

In [120]:
len(failedfinancialslist)

235

In [107]:
failedbanks.index

Int64Index([264, 266, 278, 310, 313, 328, 341, 357, 364, 395, 398, 416, 417,
            419, 422, 429, 463, 484, 488, 513, 520, 522, 571, 591, 614, 664,
            679, 680, 734, 741, 829, 836, 840, 848, 850, 852, 860, 862, 867,
            878, 884, 895, 900, 912, 916, 920, 922, 924, 932, 936],
           dtype='int64', name='CERT')

In [129]:
# failedfinancials

In [158]:
# another (faster?) approach
bankIDstring = ' OR '.join(failedbanks.index.astype(str))
failedfilter = f'CERT:({bankIDstring}) AND REPYEAR:[2005 TO 2023]'
failedfinancials2 = getData(financialsurl, failedfilter, features)

#

In [159]:
failedfinancials2

,AVASSETJ,CERT,EAMINTAN,ELNLOS,EQ,ID,LNATRESR,LNLSGRS,NAME,NIM,NTLNLS,REPYEAR,RISDATE
0,493443,9,0,-215,33724,9_20050331,1.205722,328351,UNION TRUST CO,4050,21,2005,20050331
1,514500,9,0,-215,34610,9_20050630,1.086249,342739,UNION TRUST CO,8301,42,2005,20050630
2,523842,9,0,-215,34179,9_20050930,1.081034,355863,UNION TRUST CO,12425,46,2005,20050930
3,521004,9,0,-215,34053,9_20051231,1.106170,356003,UNION TRUST CO,16340,41,2005,20051231
4,525932,9,0,0,36862,9_20060331,1.104080,364104,UNION TRUST CO,3904,-34,2006,20060331
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,150807,513,0,495,14927,513_20100331,1.303945,123643,SUMMIT BANK,1689,270,2010,20100331
1781,155970,513,0,3321,12259,513_20100630,2.198169,123600,SUMMIT BANK,3386,1998,2010,20100630
1782,159829,513,56,6948,8371,513_20100930,4.478445,122419,SUMMIT BANK,5111,2872,2010,20100930
1783,151076,513,714,8298,6216,513_20101231,3.097106,115257,SUMMIT BANK,6421,6125,2010,20101231


In [164]:
failedfinancials2.RISDATE

0       20050331
1       20050630
2       20050930
3       20051231
4       20060331
          ...   
1780    20100331
1781    20100630
1782    20100930
1783    20101231
1784    20110331
Name: RISDATE, Length: 1785, dtype: int64

In [193]:
faileddata = failedbanks.merge(failedfinancials2, left_on = ['CERT', 'targetdate'], right_on = ['CERT', 'RISDATE'], how = 'left', suffixes=['_b', '_f'])

In [195]:
faileddata.describe()

,CERT,ACTIVE,prevYr,targetdate,AVASSETJ,EAMINTAN,ELNLOS,EQ,LNATRESR,LNLSGRS,NIM,NTLNLS,REPYEAR,RISDATE
count,50.00000,50.0,50.000000,5.000000e+01,4.700000e+01,47.000000,47.000000,47.000000,47.000000,4.700000e+01,47.000000,47.000000,47.000000,4.700000e+01
mean,248.40000,0.0,2012.600000,2.012723e+07,2.223716e+05,61.914894,1959.893617,26119.021277,1.875860,1.520762e+05,7435.765957,1830.723404,2012.446809,2.012570e+07
std,133.03291,0.0,5.042675,5.042675e+04,2.385608e+05,223.989211,5411.443837,40440.924449,1.087412,2.130321e+05,8402.558503,4939.177866,5.007857,5.007857e+04
min,9.00000,0.0,2005.000000,2.005123e+07,1.646000e+04,0.000000,-827.000000,1603.000000,0.137698,1.146400e+04,802.000000,-970.000000,2005.000000,2.005123e+07
25%,160.25000,0.0,2008.000000,2.008123e+07,5.508050e+04,0.000000,27.000000,4957.500000,1.171207,2.765700e+04,1860.500000,34.000000,2008.000000,2.008123e+07
50%,226.00000,0.0,2012.500000,2.012623e+07,9.850200e+04,0.000000,200.000000,9633.000000,1.574630,6.002100e+04,3592.000000,227.000000,2012.000000,2.012123e+07
75%,353.00000,0.0,2017.750000,2.017873e+07,3.589935e+05,0.000000,937.000000,33388.500000,2.556923,2.070075e+05,10846.000000,685.000000,2016.500000,2.016623e+07
max,513.00000,0.0,2021.000000,2.021123e+07,1.108331e+06,1352.000000,31883.000000,219259.000000,6.251827,1.237150e+06,40635.000000,27530.000000,2021.000000,2.021123e+07
